In [3]:
# Imports
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import os
import sys
import keras.models
from keras.models import Sequential, Model
from keras.layers import Dense, Permute, Conv2D, MaxPooling2D, Flatten, Activation, Dropout
from keras.utils import to_categorical
from keras import backend as K
from keras.optimizers import SGD
#import tables

Using TensorFlow backend.


In [4]:
print(os.getcwd())

/Users/Andy/Andy/Undergrad/Year_1/uas/drone_code/src/vision/classifier


In [5]:
# Inputs

IMG_SIZE = 224

def letter_seg(path):
    #convert image to a numpy array
    img = Image.open(path)
    img = img.resize((IMG_SIZE,IMG_SIZE))
    img = np.array(img).astype(np.float32)
    Z = img.reshape(IMG_SIZE*IMG_SIZE,3)

    #use k-means to convert image to 2 colors
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 2
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((img.shape))

    #convert the image to black and white
    res2 = (res2 == res2[0,0]).astype(np.float32)

    #create mask for floodfilling
    h, w = res2.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    #meant to close gaps but doesn't work well
    #kernel = np.ones((14,14),np.uint8)
    #res2 = cv2.morphologyEx(res2, cv2.MORPH_CLOSE, kernel)

    #Floodfill from point (0, 0)
    cv2.floodFill(res2, mask, (0,0), 0)
    
    return np.expand_dims(res2[:,:,0], axis=2)

In [6]:
# Loading

SHAPE_INDEX = [
    'circle', 'cross', 'heptagon', 'octagon', 'pentagon', 'quarter_circle',
    'rectangle', 'semicircle', 'star', 'trapezoid', 'triangle'
]

def shape_img(path):
    img = Image.open(path)
    img = img.resize((IMG_SIZE, IMG_SIZE))
    #img = img.reshape((IMG_SIZE, IMG_SIZE))
    img = np.array(img).astype(np.float32)
    img = np.expand_dims(img, axis=0)
    return img

def letter_img(path):
    img = letter_seg(path)
    img = cv2.resize(img, dsize=(32, 32), interpolation=cv2.INTER_LINEAR)
    img = np.expand_dims(img, axis=2) 
    img = np.expand_dims(img, axis=0)
    return img

def load_model(path):
    return keras.models.load_model(path)

def predict(model, img):
    return np.argmax(model.predict(img))

def predict_shape(model, img):
    return SHAPE_INDEX[predict(model, img)]

def predict_letter(model, img):
    return chr(predict(model, img) + 65)

In [7]:
#letter_seg('../targets/output/train/00.jpg')
img = shape_img('../targets/output/train/000.jpg')
print(img.shape)

(1, 224, 224, 3)


In [8]:
# Models

def letter_model():
    K.set_image_data_format( 'channels_last' )
    model = Sequential()
    model.add(Permute((1,2, 3), input_shape=(32, 32, 1)))

    model.add(Conv2D(32, kernel_size=(5,5), activation='relu', kernel_initializer='TruncatedNormal', name="conv1"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(Dropout(.25))

    model.add(Conv2D(64, kernel_size=(5,5), activation='relu', kernel_initializer='TruncatedNormal', name="conv2"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(Dropout(.25))

    model.add(Flatten())
    model.add(Dense(1600, activation='relu', name='fc3'))
    model.add(Dense(26, activation='softmax', name='fc4'))
    return model

def shape_model():
    K.set_image_data_format( 'channels_last' )
    model = Sequential()
    model.add(Permute((1,2,3), input_shape=(IMG_SIZE, IMG_SIZE, 3)))

    model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', name="conv1_1"))
    model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', name="conv1_2"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu', name="conv2_1"))
    model.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu', name="conv2_2"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu', name="conv3_1"))
    model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu', name="conv3_2"))
    model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu', name="conv3_3"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu', name="conv4_1"))
    model.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu', name="conv4_2"))
    model.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu', name="conv4_3"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu', name="conv5_1"))
    model.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu', name="conv5_2"))
    model.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu', name="conv5_3"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Conv2D(4096, kernel_size=(7,7), activation='relu', name='fc6'))
    model.add(Conv2D(4096, kernel_size=(1,1), activation='relu', name='fc7'))
    model.add(Conv2D(13, kernel_size=(1,1),  name='fc8'))
    model.add(Flatten())
    model.add( Activation('softmax') )
    return model

In [9]:
model = shape_model()
print(model)

In [24]:
# Training

train_dir = '../targets/output/train/'
test_dir = '../targets/output/test/'

# def old_process_data(data_path, type=""):
#     if type == "":
#         raise ValueError('Please input the type of data (shape or letter)')
#     label_size = {"shape": 11, "letter": 26}
#     f = tables.open_file(data_path, mode='r')
#     x_train = f.root.train_input[()]
#     y_train = f.root.train_labels[()]
#     x_test = f.root.test_input[()]
#     y_test = f.root.test_labels[()]
#     f.close()
#     y_train = to_categorical(y_train, label_size[type])
#     y_test = to_categorical(y_test, label_size[type])
#     return x_train, y_train, x_test, y_test

def process_data(data_dir, n_images, feature="", old=False):
    if feature == "":
        raise ValueError('Please input the type of data (shape or letter)')
    label_size = {"shape": 13, "letter": 36}
    if old:
        label_size = {"shape": 11, "letter": 26}
    img_size = {"shape": (IMG_SIZE, IMG_SIZE, 3), "letter": (32, 32, 1)}
    label_col = {"shape": 5, "letter": 6}
    img_func = {"shape": shape_img, "letter": letter_img}
    # Load data from data_dir
    x_train = np.zeros((n_images,) + img_size[feature])
    path = os.path.join(data_dir, 'labels.csv')
    labels = np.loadtxt(path, delimiter=',')
    y_train = labels[0:n_images, label_col[feature]]
    for i in range(n_images):
        path = os.path.join(data_dir, '%03d.jpg'%i)
        x_train[i] = img_func[feature](path)[0,:]
    y_train = to_categorical(y_train, label_size[feature])
    return x_train, y_train

def get_train_and_test(feature, n_train, n_test):
    (x_train, y_train) = process_data(train_dir, n_train, feature)
    (x_test, y_test) = process_data(test_dir, n_test, feature)
    return x_train, y_train, x_test, y_test

In [11]:
(x_train, y_train, x_test, y_test) = get_train_and_test("shape", 1000, 200)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(1000, 224, 224, 3) (1000, 13) (200, 224, 224, 3) (200, 13)


In [74]:
def train(model, x_train, y_train, x_test, y_test, nEpochs=10, nBatch=32, model_name="model"):
    sgd = SGD(lr=0.0001)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x=x_train, y=y_train, batch_size=nBatch, epochs=nEpochs, verbose=1)
    model.evaluate(x=x_test, y=y_test, verbose=1)
#     model.save(model_name + '.hdf5')
    return model

# def process_letter(addr):
#     img = Image.open(addr).convert('L')
#     img = img.reshape((32,32))
#     img = np.array(img).astype(np.float32)
#     img = np.expand_dims(img, axis=0)
#     img = np.expand_dims(img, axis=3)
#     return img

# def process_shape(addr):
#     img = Image.open(addr).convert('L')
#     img = img.reshape((IMG_SIZE,IMG_SIZE))
#     img = np.array(img).astype(np.float32)
#     img = np.expand_dims(img, axis=0)
    
def predict(model, img):
    return model.predict(img)

def predict_shape(model, img):
    return SHAPE_INDEX[predict(model, img).argmax()]

def predict_letter(model, img):
    return chr(predict(model, img).argmax() + 65)

def load_model(path):
    return keras.models.load_model(path)

In [13]:
# img = process_letter('../targets/output/train/00.jpg')
# print(img.shape)

In [14]:
#model = train(model, x_train, y_train, x_test, y_test)
#model.evaluate(x=x_test, y=y_test)

In [17]:
shape_model = load_model('../models/shape/shape_model.h5')
letter_model = load_model('../models/letter/letter_model.hdf5')

In [26]:
old_dir = '../targets/output/old'
xs_old, ys_old = process_data(old_dir, 1000, 'shape', True)
xl_old, yl_old = process_data(old_dir, 1000, 'letter', True)
print(xs_old.shape, ys_old.shape, xl_old.shape, yl_old.shape)

(1000, 224, 224, 3) (1000, 11) (1000, 32, 32, 1) (1000, 26)


In [78]:
shape_model.evaluate(x=xs_old, y=ys_old)

1000/1000 [==============================] - 497s 497ms/step


[2.9609717235565185, 0.567]

In [79]:
letter_model.evaluate(x=xl_old, y=yl_old)

1000/1000 [==============================] - 1s 571us/step


[3.2565239391326903, 0.051]

In [80]:
i = 1
path = '../targets/output/old/%03d.jpg'%i
print(predict(shape_model, shape_img(path)))
print('predicted: ', predict_shape(shape_model, shape_img(path)))
print('true: ', SHAPE_INDEX[ys_old[i].argmax()])

[[2.1036822e-14 8.1971246e-10 1.9073529e-14 7.5558510e-14 4.8644341e-13
  2.2895628e-09 4.6307811e-09 3.5275571e-10 6.1940984e-04 3.9877923e-06
  9.9937660e-01]]
predicted:  triangle
true:  star


In [81]:
print(predict(letter_model, letter_img(path)))
print('predicted: ', predict_letter(letter_model, letter_img(path)))
print('true: ', chr(yl_old[i].argmax() + 65))

[[0.03908519 0.03751915 0.03880334 0.03896307 0.0377778  0.03828578
  0.03784261 0.03847403 0.0380136  0.0390711  0.03903157 0.03835811
  0.03882906 0.03845845 0.0384722  0.03802907 0.03949932 0.03904926
  0.03757166 0.03841253 0.03854285 0.03849417 0.03924222 0.03855116
  0.03734707 0.03827557]]
predicted:  Q
true:  U
